In [1]:
import geopandas as gp
%matplotlib inline
import shapely.geometry as shpgeo
PLACE_POLYS_MUSEUM = '../data/place_polys_museum.geojson' 
polys = gp.read_file(PLACE_POLYS_MUSEUM)

In [2]:
min_lat= 59.066574
max_lon = -8.089235 
max_lat = 50.126911
min_lon = 1.793197
lon_0 = (max_lon+min_lon)/2
lat_0 = (max_lat+min_lat)/2
eng_bbox = shpgeo.Polygon([(max_lon,min_lat), (min_lon,min_lat), (min_lon,max_lat), (max_lon, max_lat)])
polys.geometry.apply(lambda x: x.within(eng_bbox)).value_counts()

True    40
dtype: int64

In [3]:
target_crs = {'datum':'WGS84', 'no_defs':True, 'proj':'aea', 'lat_1':min_lat, 'lat_2':max_lat, 'lat_0':lat_0, 'lon_0':lon_0}
polys['area'] = polys.to_crs(crs=target_crs).geometry.apply(lambda x: x.area)

In [4]:
polys.place = polys.place.apply(lambda x: x[:-1] if x[-1].isdigit() else x)

In [9]:
polys['area'].order()

33       168.865878
19       257.431446
10       698.200966
4       1230.067669
38      1279.504821
1       1584.020382
39      2088.911301
24      2195.866266
6       2216.736539
12      2912.625841
11      3057.736830
30      3212.998709
17      3592.877985
16      3686.185964
31      3727.173238
14      3831.769639
26      3973.329678
3       4601.349663
36      5044.918191
20      5409.806191
23      5492.942587
5       5742.365189
18      5920.126294
22      6179.336180
21      6533.912401
35      6588.681236
29      7232.903487
37     11411.495578
32     11607.615542
25     12112.918051
34     12502.466091
15     13610.981613
9      19263.517111
28     25422.123650
8      33355.556802
7      33935.510206
2      43000.194671
0      54246.704706
27     69729.106205
13    949351.718350
Name: area, dtype: float64

In [12]:
places = polys.groupby('place').agg(sum).reset_index()
places.columns = ['place','area']
places.place = places.place.apply(lambda x: x.replace(' ','_'))

In [13]:
path = '../data/place_area.csv'
places.to_csv(path)

In [94]:
def haversine(lon1, lat1, lon2, lat2):
    from math import radians, cos, sin, asin, sqrt
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    m = km *1000
    return m   

def grid_line(mini, maxi, ngrid=10):
    delta = (maxi-mini)/ngrid
    return [(mini+i*delta, mini+(i+1)*delta) for i in range(ngrid)] 


def grid_bbox(sw, ne, ngrid=10):
    grid_lat = grid_line(sw[0], ne[0], ngrid)
    grid_lon = grid_line(sw[1], ne[1], ngrid)
    grids = []
    for i in range(ngrid):
        for j in range(ngrid):
            s, n = grid_lat[i]
            w, e = grid_lon[j]
            grids.append(((s,w),(n,e)))
    return grids
def area_of_grid(grid):
    (s,w), (n,e) = grid
    l = haversine(w,s,e,s)
    w = haversine(e,n,e,s)
    return l*w

def swne2poly(sw,ne):
    return shpgeo.box(sw[1], sw[0], ne[1], ne[0])

def area_poly(polygon, ngrid=100):
    w,s,e,n = polygon.bounds
    polygon.within(shpgeo.Polygon([(w,s),(e,s),(e,n),(w,n)]))
    grids = grid_bbox((s,w),(n,e),ngrid)
    unit_area = area_of_grid(grids[0])
    grids = [swne2poly(*grid)for grid in grids]
    grids = [grid for grid in grids if grid.intersects(polygon)]
    return len(grids)*unit_area